# EDA con Sweetviz

Este notebook realiza un análisis exploratorio de datos automático utilizando la librería Sweetviz para el nuevo dataset `process_data.parquet`.

In [1]:
# Instalación de dependencias
# !pip install sweetviz pandas pyarrow

In [2]:
import pandas as pd
import sweetviz as sv
import os

# Definir ruta al archivo (asumiendo ejecución desde la carpeta notebooks/)
DATA_PATH = "../data/02_intermediate/process_data.parquet"

try:
    if os.path.exists(DATA_PATH):
        df = pd.read_parquet(DATA_PATH)
        print(f"Datos cargados correctamente. Dimensiones: {df.shape}")
        
        # Mostrar las columnas para entender las variables del nuevo dataset
        print("\n--- Variables del Dataset ---")
        print(df.columns.tolist())
        print("-----------------------------\n")
    else:
        print(f"No se encontró el archivo en {DATA_PATH}. Por favor verifica que el archivo exista.")
        df = None
except Exception as e:
    print(f"Ocurrió un error al cargar los datos: {e}")
    df = None

In [3]:
if df is not None:
    # Determinar variable objetivo (ajustado para dataset 2022)
    target_col = None
    possible_targets = ['CVDINFR4', 'CVDCRHD4']

    for col in possible_targets:
        if col in df.columns:
            target_col = col
            break

    if target_col:
        print(f"Variable objetivo detectada: {target_col}")
    else:
        print("No se detectó ninguna variable objetivo conocida. Se realizará el análisis sin target.")

In [5]:
if df is not None:
    # Generar reporte
    # Agregamos pairwise_analysis='off' para que no intente cruzar todas las columnas entre sí (optimización)

    if target_col:
        # Si hay una columna objetivo definida
        report = sv.analyze(df, target_feat=target_col, pairwise_analysis='off')
    else:
        # Si no hay columna objetivo
        report = sv.analyze(df, pairwise_analysis='off')

    # Mostrar y guardar reporte
    report.show_html("SWEETVIZ_REPORT.html")
    print("Reporte generado: SWEETVIZ_REPORT.html")